# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [250]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [251]:
stocks = pd.read_csv("sp_500_stocks.csv")
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [252]:
symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'GN EAQS(/LDNTACKR) SALELOSGA METB', 'calculationPrice': 'tops', 'open': None, 'openTime': None, 'openSource': 'fcfliaio', 'close': None, 'closeTime': None, 'closeSource': 'coaiffli', 'high': None, 'highTime': None, 'highSource': None, 'low': None, 'lowTime': None, 'lowSource': None, 'latestPrice': 132.194, 'latestSource': 'IEX real time price', 'latestTime': '2:21:37 PM', 'latestUpdate': 1653163956416, 'latestVolume': None, 'iexRealtimePrice': 136.059, 'iexRealtimeSize': 103, 'iexLastUpdated': 1670477646638, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 135.29, 'previousVolume': 174610108, 'change': -0.195, 'changePercent': -0.00155, 'volume': None, 'iexMarketPercent': 0.01151835974978216, 'iexVolume': 671697, 'avgTotalVolume': 108903062, 

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [253]:
price = data["latestPrice"]
pe_ratio = data["peRatio"]
pe_ratio

42.15

## Executing A Batch API Call & Building Our DataFrame

Just like in the first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [254]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [255]:
final_df = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(","):
        final_df = final_df.append(
        pd.Series(
        [
        symbol,
        data[symbol]["quote"]["latestPrice"],
        data[symbol]["quote"]["peRatio"],
        "N/A"
        ],
        index=my_columns
        ),ignore_index=True
        )
        
        
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,118.764,55.4,N/A
1,AAL,16.603,-1.14,N/A
2,AAP,164.182,23.66,N/A
3,AAPL,133.226,41.81,N/A
4,ABBV,106.240,23.45,N/A
...,...,...,...,...
500,YUM,112.120,32.48,N/A
501,ZBH,150.600,-205.93,N/A
502,ZBRA,382.410,44.24,N/A
503,ZION,43.895,18.86,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [256]:
final_df.sort_values("Price-to-Earnings Ratio", inplace=True)
## Removing all stocks with negative P/E
final_df = final_df[final_df["Price-to-Earnings Ratio"] > 0]
final_df = final_df[:50]
final_df.reset_index(inplace=True)
## the "drop" method typically drops rows (because "axis=0" by default), so we set "axis" to equal 1, so it parses by column
final_df.drop("index", axis=1, inplace=True)
final_df.head()

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,36.340,2.19,N/A
1,NLOK,21.417,4.22,N/A
2,UNM,22.550,4.88,N/A
3,AIV,5.184,5,N/A
4,BIO,583.600,5.18,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that i created in the momentum project.

I have included this function below.

In [257]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [258]:
portfolio_input()

Enter the value of your portfolio:10000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [259]:
position_size=float(portfolio_size)/len(final_df.index)


for row in final_df.index:
    final_df.loc[row, "Number of Shares to Buy"] = math.floor(position_size/final_df.loc[row, "Price"])

final_df.head()

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,36.340,2.19,5503
1,NLOK,21.417,4.22,9338
2,UNM,22.550,4.88,8869
3,AIV,5.184,5,38580
4,BIO,583.600,5.18,342


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [260]:
symbol = "AAPL"
batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}"
data = requests.get(batch_api_call_url).json()


#Price-to-earnings ratio
pe_ratio = data[symbol]["quote"]["peRatio"]

#Price-to-book ratio
pb_ratio = data[symbol]["advanced-stats"]["priceToBook"]
#Price-to-sales ratio
ps_ratio = data[symbol]["advanced-stats"]["priceToSales"]

#Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data[symbol]["advanced-stats"]["enterpriseValue"]
ebitda = data[symbol]["advanced-stats"]["EBITDA"]
ev_to_ebitda = enterprise_value/ebitda

#Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]["advanced-stats"]["grossProfit"]
ev_to_gp = enterprise_value/gross_profit





Let's move on to building our DataFrame. The abbreviation `rv` stands for `robust value`, which is what i'll call this strategy moving forward.

In [261]:
rv_columns=[
    "Ticker",
    "Price",
    "Number of Shares to Buy",
    "P/E Ratio",
    "P/E Percentile",
    "P/B Ratio",
    "P/B Percentile",
    "P/S Ratio",
    "P/S Percentile",
    "EV/EBITDA",
    "EV/EBITDA Percentile",
    "EV/GP",
    "EV/GP Percentile",
    "RV Score"
]

rv_df = pd.DataFrame(columns=rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(","):
        ## Some metrics are missing, and that means that Python is trying to divide by/with "NoneType" with/by a number
        ## So try to calculated EV/EBITDA and EV/GP, and if there is a "TypeError", we just input "NaN" for the metric
        enterprise_value = data[symbol]["advanced-stats"]["enterpriseValue"]
        ebitda = data[symbol]["advanced-stats"]["EBITDA"]
        gross_profit = data[symbol]["advanced-stats"]["grossProfit"]
        
        
        try:
                ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
                ev_to_ebitda = np.NaN
        
        try:
                ev_to_gp = enterprise_value/gross_profit
        except TypeError:
                ev_to_gp = np.NaN
        
        rv_df = rv_df.append(pd.Series(
        [
            symbol,
            data[symbol]["quote"]["latestPrice"],
            "N/A",
            data[symbol]["quote"]["peRatio"],
            "N/A",
            data[symbol]["advanced-stats"]["priceToBook"],
            "N/A",
            data[symbol]["advanced-stats"]["priceToSales"],
            "N/A",
            ev_to_ebitda,
            "N/A",
            ev_to_gp,
            "N/A",
            "N/A"
        ]
            
        ,index = rv_columns
        ),ignore_index=True)
    
    
rv_df
    

,Ticker,Price,Number of Shares to Buy,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,119.846,N/A,54.25,N/A,7.34,N/A,7.1,N/A,34.213416,N/A,12.884051,N/A,N/A
1,AAL,16.576,N/A,-1.18,N/A,-1.76,N/A,0.3893,N/A,-6.120366,N/A,1.377920,N/A,N/A
2,AAP,162.741,N/A,23.53,N/A,3.01,N/A,1.1,N/A,11.043862,N/A,2.503844,N/A,N/A
3,AAPL,135.610,N/A,40.71,N/A,35.17,N/A,8.31,N/A,28.250060,N/A,20.974333,N/A,N/A
4,ABBV,103.800,N/A,23.51,N/A,12.29,N/A,4.61,N/A,15.289042,N/A,9.172569,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,108.780,N/A,32.25,N/A,-4,N/A,6.2,N/A,23.872679,N/A,10.421449,N/A,N/A
501,ZBH,155.310,N/A,-207.78,N/A,2.75,N/A,4.44,N/A,25.596004,N/A,7.718082,N/A,N/A
502,ZBRA,385.020,N/A,44.42,N/A,10.95,N/A,4.87,N/A,31.007558,N/A,11.108433,N/A,N/A
503,ZION,43.645,N/A,18.69,N/A,1.0149,N/A,2.38,N/A,10.462240,N/A,2.656212,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [262]:
rv_df[rv_df.isnull().any(axis=1)]
type(rv_df)

pandas.core.frame.DataFrame

Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [263]:
for column in ['P/E Ratio','P/B Ratio','P/S Ratio','EV/EBITDA', 'EV/GP']:
    rv_df[column].fillna(rv_df[column].mean(), inplace=True)
    
type(rv_df)

pandas.core.frame.DataFrame

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [264]:
rv_df.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'P/E Ratio',
       'P/E Percentile', 'P/B Ratio', 'P/B Percentile', 'P/S Ratio',
       'P/S Percentile', 'EV/EBITDA', 'EV/EBITDA Percentile', 'EV/GP',
       'EV/GP Percentile', 'RV Score'],
      dtype='object')

## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [265]:
from scipy.stats import percentileofscore as score
metrics={
'P/E Ratio':'P/E Percentile', 
'P/B Ratio':'P/B Percentile', 
'P/S Ratio':'P/S Percentile', 
'EV/EBITDA':'EV/EBITDA Percentile', 
'EV/GP':'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_df.index:
        rv_df.loc[row, metrics[metric]] = score(rv_df[metric], rv_df.loc[row, metric])/100

type(rv_df)

pandas.core.frame.DataFrame

## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [266]:
from statistics import mean

for row in rv_df.index:
    value_percentiles = [] 
    for metric in metrics.keys():
        value_percentiles.append(rv_df.loc[row, metrics[metric]])
        
    rv_df.loc[row, "RV Score"] = mean(value_percentiles)

type(rv_df)

pandas.core.frame.DataFrame

## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [267]:
rv_df.sort_values("RV Score", ascending=True, inplace=True)
rv_df = rv_df[:50]
rv_df.reset_index(inplace=True)
rv_df.drop("index", axis=1, inplace=True)
rv_df

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Ticker,Price,Number of Shares to Buy,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,22.230,N/A,4.87,0.154455,0.4084,0.049505,0.3685,0.0277228,2.316470,0.0316832,0.365561,0.0039604,0.0534653
1,AIG,38.081,N/A,-6.85,0.108911,0.5150,0.0574257,0.7241,0.0653465,4.327481,0.0415842,0.715922,0.0158416,0.0578218
2,AAL,16.576,N/A,-1.18,0.140594,-1.7600,0.0455446,0.3893,0.0316832,-6.120366,0.0277228,1.377920,0.0534653,0.059802
3,MET,47.397,N/A,7.80,0.162376,0.5630,0.0594059,0.6328,0.0554455,4.241979,0.039604,0.626488,0.0118812,0.0657426
4,HFC,26.230,N/A,-26.71,0.0673267,0.7410,0.0732673,0.2891,0.0138614,5.994011,0.0613861,2.137791,0.112871,0.0657426
5,F,9.067,N/A,-17.60,0.0792079,1.1200,0.128713,0.2738,0.0118812,4.508923,0.0435644,2.024415,0.10297,0.0732673
6,HIG,49.977,N/A,10.56,0.188119,0.9920,0.10099,0.8487,0.0910891,4.075397,0.0376238,0.818477,0.0217822,0.0879208
7,ALL,107.960,N/A,7.83,0.164356,1.2900,0.169307,0.7550,0.0712871,2.444416,0.0336634,0.694993,0.0138614,0.090495
8,BA,220.780,N/A,-28.82,0.0633663,-10.7100,0.029703,2.0400,0.352475,-29.389244,0.0158416,-204.060878,0.0019802,0.0926733
9,KSS,41.290,N/A,-59.49,0.049505,1.3000,0.173267,0.3548,0.0257426,9.568921,0.194059,1.404039,0.0574257,0.1


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [268]:
portfolio_input()

Enter the value of your portfolio:100000000


In [269]:
position_size = float(portfolio_size)/len(rv_df.index)

for row in rv_df.index:
    rv_df.loc[row, "Number of Shares to Buy"] = math.floor(position_size/rv_df.loc[row, "Price"])

rv_df

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Number of Shares to Buy,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,22.230,89968,4.87,0.154455,0.4084,0.049505,0.3685,0.0277228,2.316470,0.0316832,0.365561,0.0039604,0.0534653
1,AIG,38.081,52519,-6.85,0.108911,0.5150,0.0574257,0.7241,0.0653465,4.327481,0.0415842,0.715922,0.0158416,0.0578218
2,AAL,16.576,120656,-1.18,0.140594,-1.7600,0.0455446,0.3893,0.0316832,-6.120366,0.0277228,1.377920,0.0534653,0.059802
3,MET,47.397,42196,7.80,0.162376,0.5630,0.0594059,0.6328,0.0554455,4.241979,0.039604,0.626488,0.0118812,0.0657426
4,HFC,26.230,76248,-26.71,0.0673267,0.7410,0.0732673,0.2891,0.0138614,5.994011,0.0613861,2.137791,0.112871,0.0657426
5,F,9.067,220580,-17.60,0.0792079,1.1200,0.128713,0.2738,0.0118812,4.508923,0.0435644,2.024415,0.10297,0.0732673
6,HIG,49.977,40018,10.56,0.188119,0.9920,0.10099,0.8487,0.0910891,4.075397,0.0376238,0.818477,0.0217822,0.0879208
7,ALL,107.960,18525,7.83,0.164356,1.2900,0.169307,0.7550,0.0712871,2.444416,0.0336634,0.694993,0.0138614,0.090495
8,BA,220.780,9058,-28.82,0.0633663,-10.7100,0.029703,2.0400,0.352475,-29.389244,0.0158416,-204.060878,0.0019802,0.0926733
9,KSS,41.290,48437,-59.49,0.049505,1.3000,0.173267,0.3548,0.0257426,9.568921,0.194059,1.404039,0.0574257,0.1


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization.

In [270]:
writer = pd.ExcelWriter("Value_Strategy.xlsx", engine = "xlsxwriter")
rv_df.to_excel(writer, sheet_name="Value Strategy", index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

In [271]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            "num_format" : "0.00",
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }


    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [272]:
column_templates = {
    "A":["Ticker", string_template],
    "B":["Price", dollar_template],
    "C":["Number of Shares to Buy", integer_template],
    "D":["P/E Ratio", float_template],
    "E":["P/E Percentile", percent_template],
    "F":["P/B Ratio", float_template],
    "G":["P/B Percentile", percent_template],
    "H":["P/S Ratio", float_template],
    "I":["P/S Percentile", percent_template],
    "J":["EV/EBITDA", float_template],
    "K":["EV/EBITDA Percentile", percent_template],
    "L":["EV/GP", float_template],
    "M":["EV/GP Percentile", percent_template],
    "N":["RV Score", percent_template]
}

for column in column_templates.keys():
    writer.sheets["Value Strategy"].set_column(f"{column}:{column}", 25, column_templates[column][1])
    writer.sheets["Value Strategy"].write(f"{column}1", column_templates[column][0], column_templates[column][1])
    

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [273]:
writer.save()